<a href="https://colab.research.google.com/github/Felipehxtz/Deep-Learning-and-image-processing/blob/main/Detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam


In [ ]:
#Conecta ao google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#cria um atalho para o drive
!ln -s /content/drive/MyDrive /Mydrive
!ls /Mydrive

'Colab Notebooks'   dataset   dataset.zip


In [ ]:
#descompacta o banco de dados
%cd /Mydrive
!unzip \*.zip

/content/drive/MyDrive
Archive:  dataset.zip
replace dataset/test_set/cidade/Untitled 2 1196.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
def classificador_model():
    tf.random.set_seed(1)

    """
////////////////////////// Iniciando a Rede Neural//////////////////////////
Sequencial: Recomendada quando se tem um tensor de entrada e um de saída

Conv2D: N° de filtros, tamanho do detector de característica

Input_shape: Tamanho de entrada das imagens e suas bandas de cores.
 
Activation relu: Retira os valores negativos(partes mais escuras das imagens).

BatchNormalization: aplica uma transformação que mantém a saída média próxima 
a 0 e o desvio padrão da saída próximo a 1. Acelera o processamento.
MaxPooling2D: Reduz o overfitting, ruidos e novamente o tamanho da imagem.
Foca nas características mais relevantes.   
    """
    classificador = Sequential()
    classificador.add.(input(shape=(256, 256, 3)))
    classificador.add(Conv2D(16, (3,3), activation = 'relu'))
    classificador.add(BatchNormalization())
    classificador.add(MaxPooling2D(pool_size = (2,2)))

    """
podemos fazer o procedimento novamente, caso tivessemos imagens com muitos
pixels. A consequencia é a perda de mais características da imagem.
    """
    classificador.add(Conv2D(64, (3,3), activation = 'relu'))
    classificador.add(BatchNormalization())
    classificador.add(MaxPooling2D(pool_size = (2, 2)))
        

#Transforma a matriz em um vetor.
    classificador.add(Flatten())
    """
//////////////////////// Iniciando a Rede Neural Densa////////////////////////
Dense: N° de neurônios, função de ativação. Utilizada para configurar neurônios 
intermediários e da saída(s).
Dropout: zera uma quantidade determinada de entradas.
    """
    classificador.add(Dense(units = 128, activation = 'relu'))
    classificador.add(Dropout(0.5))
    classificador.add(Dense(units = 64, activation = 'relu'))
    classificador.add(Dropout(0.2))
    classificador.add(Dense(units = 1, activation = 'sigmoid'))
    opt = Adam(learning_rate=0.00005)  
    classificador.compile(optimizer=opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classificador

In [ ]:
"""
//////////////////////// gerando imagens para treinamento ///////////////////
Rescale: Faz a normalização dos dados, deixando-os entre 0 e 1.
Rotation_range: Grau de rotação da imagem.
Horizontal_flip: Gira as imagens na horizontal. 
shear_range: Faz a mudança dos pixels para outra direção.
Height_shift_range: Faixa de mudança da altura.
Zoom_range: Intervalo para o zoom aleatório.
"""

gerador_treinamento = ImageDataGenerator(rescale = 1./255, rotation_range = 7, 
                                         horizontal_flip = True,
                                         shear_range = 0.2,
                                         height_shift_range = 0.07,
                                         zoom_range = 0.2)

gerador_teste = ImageDataGenerator(rescale = 1./255)

"""
///////////////////// gerando classes de treinamento e teste /////////////////
Target_size: Tamanho das imagens, determinadas em input_shape.
Batch_size: Define o número de amostras que serão propagadas pela rede.
Class_mode: Define o classe de problema que temos. Neste caso temos um binário
(litoral ou cidade).
"""
base_treinamento = gerador_treinamento.flow_from_directory('/Mydrive/dataset/training_set', 
                                                           target_size = (128, 128), 
                                                           batch_size = 10, 
                                                           class_mode = 'binary')


validation_test = gerador_teste.flow_from_directory('/Mydrive/dataset/validation_test',
                                                           target_size = (128, 128),
                                                           batch_size = 10, 
                                                           class_mode = 'binary')




                          
    

Found 4500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [ ]:
import sys
print('Python version:', sys.version.split(' ')[0])
print('TensorFlow version:', tf.__version__)
print('GPU:', tf.config.list_physical_devices('GPU'))

Python version: 3.6.9
TensorFlow version: 2.4.1
GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
classificador  = classificador_model()
classificador.summary()
classificador.fit(base_treinamento, batch_size = 256, 
                            epochs = 10, validation_data = validation_test) 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 16)      448       
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 16)      64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        9280      
_________________________________________________________________
batch_normalization_1 (Batch (None, 61, 61, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 57600)             0

In [ ]:
classificador.save("/Mydrive/classificador.h5")

In [ ]:
classificador = load_model('/Mydrive/classificador.h5')

In [ ]:
base_teste = gerador_teste.flow_from_directory('/Mydrive/dataset/test_set', 
                                                           target_size = (256, 256),
                                                           batch_size = 10, 
                                                           class_mode = 'binary')
acc_test = classificador.evaluate(base_teste)